In [1]:
BASE_DIR = '../'*3
RUN_DIR = 'pass'

In [3]:
%run ../../chck_device.ipynb

Using device: cuda
GeForce GTX 1660 Ti
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [8]:
x = torch.tensor(data=[2.0,3.0],requires_grad=True)
y=x**2
z=2*y+3
for i in range(10):
    target=torch.tensor([3.0,4.0])
    loss=torch.sum(torch.abs(z-target))
    loss.backward()
    print(x.grad)

tensor([ 8., 12.])


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

C:\Users\chang\Anaconda3\envs\testtorch\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  """Entry point for launching an IPython kernel.


AttributeError: 'Tensor' object has no attribute 'grade'